# Proyecto 1 Etapa 1

## 1. Lectura y Comprensión del Enunciado

El presente modelo aborda el reto de distribución que enfrenta LogistiCo, una empresa dedicada al transporte de insumos médicos y productos esenciales en la región de La Guajira, Colombia. Este territorio se caracteriza por su geografía extensa, condiciones climáticas adversas y la dispersión de comunidades en zonas remotas, lo cual dificulta la cobertura eficiente y oportuna de la demanda.

Garantizar el acceso equitativo a recursos críticos en estas comunidades es una necesidad urgente. En este contexto, LogistiCo cuenta con una flota mixta de camionetas y drones para realizar sus entregas. Esta situación se puede abstraer como un problema de optimización, en el que se busca minimizar el costo total del servicio logístico, garantizando la cobertura completa de la demanda, el respeto por ventanas de tiempo y las capacidades físicas de los vehículos.

## 2. Identificación de Elementos Clave

- **Componentes esenciales del sistema:**

Clientes o comunidades a atender.

Flota compuesta por camionetas y drones.

Capacidad, alcance y restricciones de cada tipo de vehículo.

Tiempos máximos y mínimos de entrega (ventanas de tiempo).

Costos asociados: fijos, mantenimiento, combustible, penalizaciones.

- **Limitaciones físicas:**

Los drones tienen un alcance limitado y una capacidad de carga menor.

Las camionetas, aunque más capaces, requieren rutas terrestres transitables.

Las distancias y tiempos de viaje deben respetar las ventanas de servicio del cliente.

- **Limitaciones organizacionales:**

Los recursos logísticos (vehículos) son limitados.

Se prioriza la minimización de costos, sin sacrificar la calidad del servicio.

Penalizaciones económicas si se incumplen ventanas de tiempo.

## 3. Formulación de la Estructura del Modelo

---

#### **Conjuntos**

- **𝐼**: Conjunto de clientes o comunidades.  
- **𝑉**: Conjunto de vehículos disponibles.

---

#### **Parámetros**

$$
\text{Dist}_{iv} = \text{Distancia del depósito al cliente } i \text{ usando el vehículo } v
$$

$$
F_t = \text{Tarifa fija de flete}
$$

$$
C_m = \text{Costo de mantenimiento por kilómetro}
$$

$$
P_f = \text{Precio del combustible}
$$

$$
R_v = \text{Rendimiento de combustible del vehículo } v
$$

$$
T_{\min,i}, T_{\max,i} = \text{Ventana de tiempo de atención del cliente } i
$$

$$
\text{Demanda}_i = \text{Cantidad de insumos que requiere el cliente } i
$$

$$
\text{Cap}_v = \text{Capacidad de carga del vehículo } v
$$

$$
\text{Rango}_v = \text{Rango máximo que puede recorrer el vehículo } v
$$

---

#### **Variables de Decisión**

- **𝑥ᵢᵥ ∈ {0,1}**: Indica si el vehículo *v* atiende al cliente *i*.  


---

#### **Variables Auxiliares**

- **𝛿ᵥ**: Toma valor 1 si el vehículo *v* es camioneta, y 0 si es dron.

---


### Formulación de Restricciones:

#### **Ventanas de Tiempo:**

$T_{min,i} \leq t_{iv} \leq T_{max,i} \quad \forall i \in I,\; \forall v \in V$

#### **Capacidad de los Vehículos:**

$\sum_{i \in I} Demanda_i \cdot x_{iv} \leq Cap_v \quad \forall v \in V$

#### **Autonomía de los Vehículos:**

$\sum_{i \in I} Dist_{iv} \cdot x_{iv} \leq Rango_v \quad \forall v \in V$

#### **Asignación:** (DUDAA)

$\sum_{v \in V} x_{iv} = 1 \quad \forall i \in I$

En términos de que un cliente solo puede ser atendido por un vehículo para una misión específica

DUDA: Deberia haber una restricción tipo la suma de la variable de decisión debe ser mayor a 1 para un día, o sea en un día deben llevarle a 3 clientes...minimo que se usen 2 vehiculos

#### **Restricción por condiciones climáticas, de geografía, etc.**
Matriz binaria $Accesibilidad_{iv}$:

- $Accesibilidad_{iv} = 1$ si el vehículo $v$ puede acceder al cliente $i$  
- $Accesibilidad_{iv} = 0$ si no puede (por clima, geografía, etc.)

$$
x_{iv} \leq Accesibilidad_{iv} \quad \forall i \in I,\ \forall v \in V
$$

**Ejemplos:**

- Si hay una montaña entre el cliente y el depósito, $Accesibilidad_{iv} = 0$ para **camionetas**.
- Si el clima impide volar, $Accesibilidad_{iv} = 0$ para **drones** ese día.

#### **Retorno para reabastecer (si no cabe toda la carga)**

Si la demanda de múltiples clientes excede la capacidad de un vehículo, se permite que:

- **El mismo vehículo realice varios viajes.**
- **Pero un cliente solo puede ser atendido una vez** (por un solo viaje de algún vehículo).

Para evitar que el mismo cliente sea visitado más de una vez:

$$
x_{iv} =
\begin{cases}
1 & \text{si el vehículo } v \text{ atiende al cliente } i \\
0 & \text{en otro caso}
\end{cases}
$$

Y se impone:

$$
\sum_{v \in V} x_{iv} = 1 \quad \forall i \in I
$$

Lo cual garantiza que **cada cliente sea atendido por un solo vehículo en un solo viaje**.

## 4. Preprocesamiento de datos

In [ ]:
origen = {
    0: (-72.9071, 11.5449)
}

destino = {
    1: (-72.8500, 11.6000),
    2: (-72.9200, 11.5200),
    3: (-72.8800, 11.5800),
}

In [6]:
import requests

# Tu API key
api_key = '5b3ce3597851110001cf6248be808c25e2964fb2b3e74f3c74d8ce63'

# Coordenadas de origen y destino (formato "longitud,latitud")
origen = "-72.9071, 11.5449"
destino = "-72.8500, 11.6000"

# Endpoint para rutas en vehículo (driving-car)
url = 'https://api.openrouteservice.org/v2/directions/driving-car'

# Define los encabezados y parámetros
headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}
params = {
    'start': origen,
    'end': destino
}

# Realiza la solicitud GET a la API
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    segmento = data['features'][0]['properties']['segments'][0]
    distancia = segmento['distance']  # en metros
    duracion = segmento['duration']    # en segundos
    print(f"Distancia: {distancia/1000:.2f} km")
    print(f"Duración: {duracion/60:.2f} minutos")
else:
    print("Error en la solicitud:", response.status_code, response.text)


Distancia: 21.08 km
Duración: 27.03 minutos


## 5. Desarrollo de la Función Objetivo

#### ¿Qué buscamos minimizar?

Como podemos observar el problema nos proporciona diferentes objetivos de optimizacion como lo es la distancia, el tiempo o costo. Teniendo en cuenta que estamos bajo un contexto donde es indispensable la llegada de las flotas a las diferentes zonas de la Guajira se propone que el mejor enfoque de optimizacion es para los costos, esto entiende todos aquellos costos operativos incluidos en la acció.

1. **Costo por distancia**:
$$
\text{Distancia} \times (\text{Tarifa de flete} + \text{Costo de mantenimiento})
$$

2. **Costo por combustible** (solo para camionetas):
$$
\frac{\text{Distancia}}{\text{Rendimiento}} \times \text{Precio del combustible}
$$

3. **Penalización por tiempo de entrega excedido**:

Se aplica una penalización económica cuando un pedido es entregado fuera de su ventana de tiempo, específicamente después del tiempo máximo permitido. Esta penalización se define como:

$$
\left( \frac{\text{Retraso (min)}}{10} \right) \times 10{,}000
$$

donde el retraso en minutos se divide entre 10 y se multiplica por un valor penalizador de $10{,}000$ unidades monetarias, incentivando así la puntualidad en las entregas.


---

### Formulación Simplificada de la Función Objetivo

$$
\min \sum_{v \in V} \sum_{i \in I} \sum_{j \in I} 
\Big( P_f \cdot Dist_{iv} \cdot 𝛿ᵥ + C_m + 𝛿ᵥ \cdot C_e + F_t + C_v \cdot t_{ij} \Big) x_{ijv} 
$$

donde:

- $x_{iv}$: variable binaria que vale 1 si el vehículo $v$ atiende al cliente $i$, y 0 en caso contrario.  
- $\text{Dist}_{iv}$: distancia entre el depósito y el cliente $i$ cuando es atendido por el vehículo $v$.  
- $Ft$: tarifa de operación fija por kilómetro ($5{,}000$ unidades monetarias).  
- $Cm$: costo de mantenimiento por kilómetro ($700$ unidades monetarias).  
- $Pf$: precio del combustible ($15{,}000$ unidades monetarias por litro).   
- $\delta_v$: parámetro que vale 1 si el vehículo $v$ es una camioneta, y 0 si es un dron (ya que los drones no consumen combustible). 
- $C_v$ es el costo por viaje
- $t_{ij}$ es el tiempo de viaje entre $i$ y $j$. 
- $C_e$ es el costo electrico

La formulación de la función objetivo se basa en los parámetros, conjuntos y variables previamente definidos. Para simplificar el ejercicio y cumplir con su propósito, todas las variables se expresan en términos de costos. Nuestra función objetivo permite evaluar escenarios en los que se utilice un dron o un vehículo todoterreno.  

Se decidió no incluir penalizaciones directamente en la función objetivo, sino considerarlas únicamente como restricciones. Esto se debe a que los productos a entregar son críticos y deben ser entregados sin excepción.

---

### Ejemplo de cálculo del costo para un cliente con camioneta

**Datos de entrada**:
- Distancia del depósito a Caserío El Sol = \(10 \, \text{km}\)
- Vehículo asignado: camioneta (200 unidades de capacidad, 300 km de rango)
- Envío se realiza 15 minutos después del máximo permitido
- Rendimiento de la camioneta: \(5 \, \text{km/L}\)

**Costo fijo por distancia**:
$$
10 \cdot (5{,}000 + 700) = 10 \cdot 5{,}700 = 57{,}000
$$

**Costo de combustible**:
$$
\left( \frac{10}{5} \right) \cdot 15{,}000 = 2 \cdot 15{,}000 = 30{,}000
$$

**Penalización por retraso**:
$$
\left( \frac{15}{10} \right) \cdot 10{,}000 = 1.5 \cdot 10{,}000 = 15{,}000
$$

**Costo total**:
$$
57{,}000 + 30{,}000 + 15{,}000 = 102{,}000
$$



---

## 6. Validación y Análisis

## 7.Ejemplos ilustrativos